In [1]:
import tensorflow as tf 
from tensorflow.contrib import rnn 
from tensorflow.examples.tutorials.mnist import input_data
# we download a data set
# one_hot - one file is taken and is hot, rest is cold so ofline 
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# number of classes
n_classes = 10
batch_size = 64
learning_rate=0.001
hm_epochs = 3
chunks_size = 28
n_chunks = 28
rnn_size = 128

In [3]:
x=tf.placeholder('float32',[None,n_chunks, chunks_size])
y=tf.placeholder('float32')
# my own function wyth low level layers
def rnn_model(x):
    layer={'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
            'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunks_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size) 
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output= tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    return output

In [4]:
def train_rnn(x):
    prediction=rnn_model(x)
    cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss=0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunks_size))
                _,c=sess.run([optimizer,cost],feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                #print('epoch ', epoch, 'complited out of ', hm_epochs, 'loss ', epoch_loss)
            
        correct= tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy= tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunks_size)), y:mnist.test.labels}))

In [5]:
train_rnn(x)

Accuracy: 0.976
